# C4.5

<br>

## Introduction

<br>
Practical issues in learning decision trees include determining how deeply to grow the decision tree, handling continuous attributes, choosing an appropriate attribute selection measure, handling training data with missing attribute values, handling
attributes with differing costs, and improving computational efficiency. 

<br>
Below we discuss each of these issues and the corresponding extensions to the basic ID3 algorithm, which has been further developed to address most of these issues, with the resulting system renamed C4.5 (<a href="https://bit.ly/2HIsjwG">Quinlan, 1993</a>)


## [I1] Avoiding Overfitting the Data

<br>
The ID3 algorithm grows each branch of the tree just deeply enough to perfectly classify the training examples. While this is sometimes a reasonable strategy, in fact it can lead to difficulties when there is noise in the data, or when the number of training examples is too small to produce a representative sample of the true target function. In either of these cases, this simple algorithm can produce trees that overfit the training examples.

<br>
Given a hypothesis space $ \ H \ $, a hypothesis $ \ h \ \in \ H \ $ is said to overfit the training data if there exists some alternative hypothesis $ \ h' \ \in \ H \ $, such that $ \ h \ $ has smaller error than $ \ h' \ $ over the training examples, but the latter outperforms the former over the entire distribution of instances.


### Overfitting due to noise

<br>
One way overfitting can occur is when the training examples contain random errors or noise. To illustrate this, let's consider the effect of adding this extra positive training example, incorrectly labeled as negative, to the original collection : 


<a id='table_training_set'></a>

| #  | Outlook  | Temperature | Humidity | Windy | Play  |
| --:|:-------- |:----------- |:-------- |:----- |:-----:|
|  1 | sunny    | hot         | high     | false | N     |
|  2 | sunny    | hot         | high     | true  | N     |
|  3 | overcast | hot         | high     | false | P     |
|  4 | rain     | mild        | high     | false | P     |
|  5 | rain     | cool        | normal   | false | P     |
|  6 | rain     | cool        | normal   | true  | N     |
|  7 | overcast | cool        | normal   | true  | P     |
|  8 | sunny    | mild        | high     | false | N     |
|  9 | sunny    | cool        | normal   | false | P     |
| 10 | rain     | mild        | normal   | false | P     |
| 11 | sunny    | mild        | normal   | true  | P     |
| 12 | overcast | mild        | high     | true  | P     |
| 13 | overcast | hot         | normal   | false | P     |
| 14 | rain     | mild        | high     | true  | N     |
| **15** | **sunny** | **hot** | **normal** | **strong** | **N** |

<br>
<center><b>Table 1</b> : a small training set</center>

<br>
Given the original error-free data, ID3 will produce the decision tree shown in the picture below. However, the addition of this incorrect example will now cause ID3 to construct a more complex tree; in particular, the new example will be sorted into the second leaf node from the left in the learned tree, along with the previous positive examples no. 9 and 11. 


<br>
<a id='decision_trees_mitchell_outlook'>
    <img src="images/mitchell_outlook.jpg" alt="outlook" width="50%" height="50%">
</a>

<br>
<center><b>Figure 1</b> : a decision tree for the concept PlayTennis</center>

<br>
Because the new example is labeled as a negative example, ID3 will search for further refinements to the tree below this node. Of course, as long as the new erroneous example differs in some arbitrary way from the other examples affiliated with this node, ID3 will succeed in finding a new decision attribute to separate out this new example from the two previous positive examples at this tree node. 

<br>
The result is that ID3 will output a decision tree $ \ h \ $ that is more complex than the original tree $ \ h' \ $. Of course the new tree will fit the collection of training examples perfectly, whereas the simpler tree will not. However, given that the new decision node is simply a consequence of fitting the noisy training example, we expect $ \ h' \ $ to outperform $ \ h \ $ over subsequent data drawn from the same instance distribution.

### Overfitting due to small size of terminal nodes

<br>
Overfitting can also occur when the terminal (leaf) nodes are associated with a small numbers of examples. In this case, it is quite possible for coincidental regularities to be found, in which some attribute happens to partition the examples very well, despite being unrelated to the actual target function. Whenever such coincidental regularities exist, there is a risk of overfitting.


<br>
<a id='decision_trees_mitchell_outlook'>
    <img src="images/mitchell_outlook.jpg" alt="outlook" width="50%" height="50%">
</a>

<br>
<center><b>Figure 1</b> : a decision tree for the concept PlayTennis</center>

### Solutions

<br>
There are several approaches to avoiding overfitting in decision tree learning which can be grouped into two classes :

<br>
<ul style="list-style-type:square">
    <li>
         approaches that <b>stop growing the tree</b> earlier, <b>before it reaches the point where it perfectly classifies the
         training data</b>
    </li>
    <br>
    <li>
        approaches that allow the tree to overfit the data, and then <b>post-prune the tree</b>
    </li>
</ul>

<br>
Although the first of these approaches might seem.more direct, the second approach of <b>post-pruning overfit trees has been found to be more successful in practice</b>. This is due to the difficulty in the first approach of estimating precisely when to stop growing the tree.


#### [ S1 ] Reduced-Error Pruning

<br>
Reduced-error pruning (Quinlan, 1987) considers each of the decision nodes in the tree to be candidates for pruning. Pruning a decision node consists of <b>removing the subtree rooted at that node, making it a leaf node, and assigning it the most common classification</b> of the training examples <b>affiliated with that node</b>. 

<br>
Nodes are removed only if the resulting pruned tree performs no worse than the original over the validation set. This has the effect that <b>any leaf node added due to coincidental regularities in the training set is likely to be pruned because these
same coincidences are unlikely to occur in the validation set</b>. 

<br>
<b>Figure 2</b> illustrates the impact of overfitting in a typical application of decision tree learning: as ID3 adds new nodes to grow the decision tree, the accuracy of the tree measured over the training examples increases monotonically. However, when measured over a set of test examples independent of the training examples, accuracy first increases, then decreases.

<table>
    <tr>
        <td width="49.5%"><img src="images/mitchell_overfitting.png" alt="overfitting"></td>
        <td width="50.5%"><img src="images/mitchell_reduced_error_pruning.jpg" alt="reduced error pruning"></td>
    </tr>
    <tr>
        <th><center><b>Figure 2</b> : overfitting</center></th>
        <th><center><b>Figure 3</b> : reduced error pruning</center></th>
    </tr>    
</table>
<br>

<b>Figure 3</b> shows the same curves of training and test set accuracy as in Figure 2; the additional line shows the impact of reduced error pruning of the tree produced by ID3. When pruning begins, the tree is at its maximum size and lowest accuracy over the test set. Nodes are pruned iteratively, always choosing the node whose removal most increases the decision tree accuracy over the validation set. As pruning proceeds, the number of nodes is reduced and accuracy over the test set increases. <b>Pruning of nodes continues until further pruning is harmful (i.e. decreases accuracy of the tree over the validation set)</b>.

<br>
Here, the available data has been split into three subsets: the training examples, the validation examples used for pruning the tree, and a set of test examples used to provide an unbiased estimate of accuracy over future unseen examples.

#### [ S2 ] Rule Post Pruning

<br>
One quite successful method for finding high accuracy hypotheses is a technique we shall call <b>rule post-pruning</b>, a variant of which is implemented in C4.5 (<a href="https://bit.ly/2HIsjwG">Quinlan, 1993</a>). Rule post-pruning involves the following steps :

<br>
<ol type="i">
    <li>
        infer the decision tree from the training set, <b>growing the tree until the training data is fit as well as
        possible</b> and allowing overfitting to occur
    </li>
    <br>
    <li>
        <b>convert the learned tree into an equivalent set of rules</b> by creating one rule for each path from the root node to
        a leaf node
    </li>
    <br>
    <li>
        <b>prune (generalize) each rule</b> by removing any preconditions whose removal does not worsen the estimated rule
        accuracy
    </li>
    <br>
    <li>
        <b>sort the pruned rules by their estimated accuracy</b>, and consider them in this sequence when classifying subsequent
        instances
    </li>
</ol>

<br>
To illustrate, let's consider again the decision tree in the <a href='#decision_trees_mitchell_outlook'>first figure</a>. In rule post-pruning, one rule is generated for each leaf node in the tree. Each attribute test along the path from the root to the leaf becomes a rule antecedent (or precondition) and the classification at the leaf node becomes the rule consequent (or post-condition). 

<br>
For example, the leftmost path of the tree is translated into the rule :

<br>
$
    \quad
    \begin{align}
        \text{if } \quad & \text{(Outlook = Sunny) } \land \text{ (Humidity = High)}
        \newline
        \text{then } \quad & \text{PlayTennis = No}
    \end{align}
$

<br>
Next, each such rule is pruned by removing any antecedent, or precondition, whose removal does not worsen its estimated accuracy. Given the above rule, for example, rule post-pruning would consider removing the preconditions $\text{(Outlook = Sunny)}$ and $\text{(Humidity = High)}$. It would select whichever of these pruning steps produced the greatest improvement in estimated rule accuracy, then consider pruning the second precondition as a further pruning step. No pruning step is performed if it reduces the estimated rule accuracy.

<br>
<b>Why convert the decision tree to rules before pruning</b>? There are three main advantages :

<br>
<ul style="list-style-type:square">
    <li>
        <b>converting to rules allows distinguishing among the different contexts in which a decision node is used</b>. Because
        each distinct path through the decision tree node produces a distinct rule, the pruning decision regarding that
        attribute test can be made differently for each path. In contrast, if the tree itself were pruned, the only two choices
        would be to remove the decision node completely, or to retain it in its original form
    </li>
    <br>
    <li>
        <b>converting to rules removes the distinction between attribute tests that occur near the root of the tree and those
        that occur near the leaves</b>. Thus, we avoid messy bookkeeping issues such as how to reorganize the tree if the root
        node is pruned while retaining part of the subtree below this test
    </li>
    <br>
    <li>
        <b>converting to rules improves readability</b>; rules are often easier for people to understand
    </li>
</ul>


## [I2] Incorporating Continuous-Valued Attributes

<br>
The initial definition of ID3 is restricted to attributes that take on a discrete set of values. Both the target and the decision attributes (those which are tested in the decision nodes) must be discrete-valued. 


### Binary Discretization

<br>
The second restriction can easily be removed so that continuous-valued decision attributes can be incorporated into the learned tree. This can be accomplished by dynamically defining new discrete-valued attributes that partition the continuous attribute value into a discrete set of intervals. In particular, for a continuous-valued attribute $ \ A \ $, the algorithm can dynamically create a new boolean attribute $ \ A' \ $, that is true if $ \ A < c \ $ and false otherwise. The only question is how to select the best value for the threshold $ \ c \ $.

<b>Example</b> : suppose we wish to include the continuous-valued attribute $ \ Temperature \ $ in describing the decision tree we are already familiar with (<a href='#decision_trees_mitchell_outlook_2'>figure</a>); suppose further that the training examples associated with a particular node in the decision tree have the following values for $ \ Temperature \ $ and the target attribute $ \ PlayTennis \ $ :

<br>
<a id='decision_trees_mitchell_continuous_values'>
    <img src="images/mitchell_continuous_values.jpg" alt="continuous_values" width="40%" height="40%">
</a>

<br>
Clearly, we would like to pick a threshold $ \ c \ $ that produces the greatest information gain. <b>By sorting the examples according to the continuous attribute $ \ A \ $, then identifying adjacent examples that differ in their target classification, we can generate a set of candidate thresholds midway between the corresponding values of $ \ A \ $. These candidate thresholds
can then be evaluated by computing the information gain associated with each</b>.

<br>
In the current example, there are two candidate thresholds, corresponding to the values of $ \ Temperature \ $ at which the value of $ \ PlayTennis \ $ changes: (48 + 60)/2 and (80 + 90)/2. The information gain can then be computed for each of the
candidate attributes, $ \ Temperature_{ \ > 54} \ $ and $ \ Temperature_{ \ > 85} \ $, and the best can be selected.

<br>
The method described above dynamically creates a boolean attribute which can then compete with the other discrete-valued decision attributes available for growing the decision tree.


### Generalizations

<br>
Fayyad and Irani (<a href="https://go.nasa.gov/2spA6Ks">Multi-Interval Discretization of Continuous-Valued Attributes for Classification Learning, 1993</a>) discuss an extension to this approach that splits the continuous attribute into multiple intervals rather than just two intervals based on a single threshold.

<br>
Utgoff and Brodley (<a href="https://bit.ly/2K2NADv">Linear machine decision trees, 1991</a>) and Murthy et al. (<a href="https://bit.ly/2I7sge7">A System for Induction of Oblique Decision Trees, 1994</a>) discuss approaches that define features by thresholding linear combinations of several continuous-valued attributes.


## [I3] Alternative Measures for Selecting Attributes

<br>
<b>There is a natural bias in the information gain measure that favors attributes with many values over those with few values</b> : a decision attribute with a large number of values is bound to separate the training examples into very small subsets and, because of this, it will have a very high information gain relative to the training examples (despite being a very poor predictor of the target function over unseen instances).

<br>
Such attribute would then be selected as the decision attribute for the root node of the tree and lead to a (quite
broad) tree of depth one, which perfectly classifies the training data while performing poorly on subsequent examples. One way to avoid this difficulty is to select decision attributes based on some measure other than information gain. 

<br>
One alternative measure that has been used successfully is the Gain Ratio (<a href="https://bit.ly/2JCjrKM">Quinlan, 1986</a>). <b>The gain ratio penalizes attributes with a large number of values by incorporating a term, called Split Information, that is sensitive to how broadly and uniformly the attribute splits the data</b> or, in other words, discourages the selection of attributes with many uniformly distributed values :

<br>
$
    \quad
    \operatorname{Split Information} \ (S, A) \quad = \quad
        - \sum _{i=1}^{c} \dfrac{\mid \ S_i \ \mid}{\mid \ S \ \mid} \ \log _{2} \dfrac{\mid \ S_i \ \mid}{\mid \ S \ \mid}
        \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \quad
        [\textbf{E3}] 
$

where $ \ S_1 \ $ through $ \ S_c \ $ are the subsets resulting from partitioning the original collection by attribute $ \ A \ $. 

<br>
Split Information is actually the entropy of $ \ S \ $ with respect to the values of attribute $ \ A \ $; this is in contrast to our previous uses of entropy, in which we considered only the entropy of $ \ S \ $ with respect to the target attribute.

<br><br>
The Gain Ratio measure is defined in terms of both Information Gain and Split Information as follows :

<br>
$
    \quad
    \operatorname{Gain Ratio} \ (S, A) \quad = \quad
        \dfrac {\operatorname{Gain} \ (S, A)} {\operatorname{Split Information} \ (S, A)}
        \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \qquad \quad
        [\textbf{E4}] 
$

<br>
### Limitations of Gain Ratio

<br>
One practical issue that arises in using gain ratio in place of gain to select attributes is that the denominator can be zero or very small when $ \ S_i \simeq S \ $ for one of the subsets; this either makes the <b>gain ratio undefined or very large for
attributes that happen to have the same value for nearly all members of the original collection</b>. 

<br>
To avoid selecting attributes purely on this basis, we can adopt some heuristic such as first calculating the Gain of each attribute, then applying the gain ration test only considering those attributes with gain above the average (<a href="https://bit.ly/2JCjrKM">Quinlan, 1986</a>). Further selection methods are discussed in <a href="https://bit.ly/2IYw6Yc">Tom Mitchell's Machine Learning</a>.

## [I4] Missing Data

<br>
In certain cases, the available data may be missing values for some attributes. In such cases, it is common to estimate the missing attribute value based on other examples for which this attribute has a known value.

<br>
Let's consider the situation in which $ \ Gain(S, A) \ $ is to be calculated at node $ \ n $ in the decision tree to evaluate whether the attribute $ \ A \ $ is the best attribute to test at this decision node; also suppose that $ \ (x, c(x)) \ $ is one of the training examples in $ \ S \ $ and that the value $ \ A(x) \ $ is unknown.

<br>
### Solutions

<br>
One strategy for dealing with the missing attribute value is to assign it the value that is most common among training examples at node $ \ n \ $. Alternatively, we might assign it the most common value among examples at node $ \ n \ $ that have the
classification $ \ c(x) \ $. The elaborated training example using this estimated value for $ \ A(x) \ $ can then be used directly by the existing decision tree learning algorithm. This strategy is examined by Mingers (<a href="https://bit.ly/2lq4YHH">An Empirical Comparison of Selection Measures for Decision-Tree Induction, 1989</a>).

<br>
A second, more complex procedure is to assign a probability to each of the possible values of $ \ A \ $, rather than simply assigning the most common value to $ \ A(x) \ $; these probabilities can be estimated again based on the observed frequencies of
the various values for $ \ A(x) \ $ among the examples at node $ \ n \ $. 

<br>
#### Example

<br>
Given a boolean attribute $ \ A \ $, if node $ \ n \ $ contains six known examples with $ \ A = 1 \ $ and four with $ \ A = 0 \ $, then we would say the probability that $ \ A(x) = 1 $ is 0.6, and the probability that $ \ A(x) = 0 \ $ is 0.4. A fractional 0.6 of instance $ \ x \ $ is now distributed down the branch for $ \ A = 1 \ $, and a fractional 0.4 of $ \ x \ $ down the other tree branch. 

<br>
These fractional examples are used for the purpose of computing Information Gain and can be further subdivided at subsequent branches of the tree if a second missing attribute value must be tested. This same fractioning of examples can also be applied after learning, to classify new instances whose attribute values are unknown. In this case, the classification of the new instance is simply the most probable classification, computed by summing the weights of the instance fragments classified in different ways at the leaf nodes of the tree. This method for handling missing attribute values is used in C4.5 (<a href="https://bit.ly/2HIsjwG">Quinlan, 1993</a>).

## Considerations

<br>
Decision trees provide a practical method for learning <b>discrete-valued functions</b>. The ID3 family of algorithms infers decision trees by growing them from the root downward, greedily selecting the next best attribute for each new decision branch added to the tree.

<br>
ID3 searches a <b>complete hypothesis space (i.e. the space of decision trees can represent any discrete-valued function defined over discrete-valued instances)</b>; it thereby avoids the major difficulty associated with approaches that consider only restricted sets of hypotheses : that the target function might not be present in the hypothesis space.

<br>
The <b>inductive bias implicit in ID3 includes a preference for smaller trees</b>; its search through the hypothesis space grows the tree only as large as needed in order to classify the available training examples.

<br>
Overfitting the training data is an important issue in decision tree learning; <b>post-pruning methods are therefore important to avoid overfitting in decision tree learning</b> (and other inductive inference methods that employ a preference bias).

<br>
A large variety of <b>extensions to the basic ID3 algorithm has been developed</b> by different researchers. These include methods for post-pruning trees, handling real-valued attributes, accommodating training examples with missing attribute values, incrementally refining decision trees as new training examples become available, using attribute selection measures other than
information gain, and considering costs associated with instance attributes.


## References

<br>
<ul style="list-style-type:square">
    <li>
         Tom Mitchell - 
         <a href="https://bit.ly/2IYw6Yc">
         Machine Learning</a>        
    </li>
    <br>
    <li>
        J. Ross Quinlan - 
        <a href="https://bit.ly/2JCjrKM">
        Induction of Decision Trees</a>  
    </li>
    <br>
    <li>
         Fayyad and Irani - 
         <a href="https://go.nasa.gov/2spA6Ks">
         Multi-Interval Discretization of Continuous-Valued Attributes for Classification Learning</a>        
    </li>
    <br>
    <li>
         Utgoff and Brodley - 
         <a href="https://bit.ly/2K2NADv">
         Linear machine decision trees</a>        
    </li>
    <br>
    <li>
         Murthy et al - 
         <a href="https://bit.ly/2I7sge7">
         A System for Induction of Oblique Decision Trees</a>        
    </li>
    <br>
    <li>
        Mingers - 
        <a href="https://bit.ly/2lq4YHH">
        An Empirical Comparison of Selection Measures for Decision-Tree Induction</a>
    </li>
    <br>
    <li>
        Quinlan - 
        <a href="https://bit.ly/2HIsjwG">
        C4.5: Programs for Machine Learning</a>
    </li>
</ul>